In [2]:
from pathlib import Path
from tqdm import tqdm
import os
import numpy as np
import json
import urllib
import PIL.Image as Image
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

In [3]:
data_dir = "COCO_Data//"
train_dir = data_dir + "train//"

train_labels = []
with open(train_dir+"labels.json") as f:
    for lines in f:
        train_labels.append(json.loads(lines))
        
train_labels_mod = dict(train_labels[0])

In [4]:
train_labels_mod['annotations']

[{'segmentation': [[392.54,
    324.66,
    351.22,
    324.66,
    337.45,
    335.48,
    324.66,
    327.61,
    339.41,
    315.8,
    392.54,
    305.96,
    396.47,
    288.26,
    398.44,
    247.92,
    415.17,
    267.6,
    429.92,
    271.53,
    456.49,
    254.81,
    459.44,
    277.43,
    518.47,
    281.37,
    553.88,
    294.16,
    574.54,
    332.53,
    575.53,
    366.96,
    549.95,
    397.46,
    529.29,
    426.97,
    514.53,
    428.94,
    519.45,
    395.49,
    490.92,
    416.15,
    467.31,
    408.28,
    507.64,
    387.62,
    496.82,
    364.99,
    471.24,
    376.8,
    462.39,
    365.98,
    462.39,
    365.98,
    453.53,
    360.07,
    425.99,
    354.17,
    381.72,
    359.09,
    347.28,
    362.04,
    337.45,
    353.19,
    356.14,
    344.33,
    402.38,
    344.33,
    393.52,
    320.72]],
  'area': 19957.726749999994,
  'iscrowd': 0,
  'image_id': 16164,
  'bbox': [324.66, 247.92, 250.87, 181.02],
  'category_id': 18,
  'id': 1767}

In [6]:
label_ids = set()
for items in train_labels_mod['annotations']:
    label_ids.add(items['category_id'])

In [11]:
x = set(range(1,91))

In [15]:
print(len(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eyeglasses' ,'handbag', 'tie', 
        'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard','tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk' 'toilet', 'door', 'tv', 'laptop',
         'mouse', 'remote', 'keyboard', 'cell phone','microwave', 'oven', 'toaster', 'sink', 'refrigerator','blender','book', 'clock', 'vase', 'scissors', 'teddy bear','hair drier', 'toothbrush']  ))

89


In [20]:
cats = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eyeglasses' ,'handbag', 'tie', 
        'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard','tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk' ,'toilet', 'door', 'tv', 'laptop',
         'mouse', 'remote', 'keyboard', 'cell phone','microwave', 'oven', 'toaster', 'sink', 'refrigerator','blender','book', 'clock', 'vase', 'scissors', 'teddy bear','hair drier', 'toothbrush']

In [21]:
m = enumerate(cats)

In [22]:
print(list(m))

[(0, 'person'), (1, 'bicycle'), (2, 'car'), (3, 'motorcycle'), (4, 'airplane'), (5, 'bus'), (6, 'train'), (7, 'truck'), (8, 'boat'), (9, 'traffic light'), (10, 'fire hydrant'), (11, 'street sign'), (12, 'stop sign'), (13, 'parking meter'), (14, 'bench'), (15, 'bird'), (16, 'cat'), (17, 'dog'), (18, 'horse'), (19, 'sheep'), (20, 'cow'), (21, 'elephant'), (22, 'bear'), (23, 'zebra'), (24, 'giraffe'), (25, 'hat'), (26, 'backpack'), (27, 'umbrella'), (28, 'shoe'), (29, 'eyeglasses'), (30, 'handbag'), (31, 'tie'), (32, 'suitcase'), (33, 'frisbee'), (34, 'skis'), (35, 'snowboard'), (36, 'sports ball'), (37, 'kite'), (38, 'baseball bat'), (39, 'baseball glove'), (40, 'skateboard'), (41, 'surfboard'), (42, 'tennis racket'), (43, 'bottle'), (44, 'plate'), (45, 'wine glass'), (46, 'cup'), (47, 'fork'), (48, 'knife'), (49, 'spoon'), (50, 'bowl'), (51, 'banana'), (52, 'apple'), (53, 'sandwich'), (54, 'orange'), (55, 'broccoli'), (56, 'carrot'), (57, 'hot dog'), (58, 'pizza'), (59, 'donut'), (60, '